In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
import random
import csv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pickle
import sys
from utils import utils
import numpy as np

In [ ]:
#g = nx.read_edgelist("data/decoded.tsv")

#print(nx.info(g))
pd.read_csv('data/decoded.tsv', nrows=80, delimiter='\t').to_csv('data/decodedSample.tsv', sep='\t', header = False, index = False)

#reading the edgelist in a variable using networkX
subgraph = nx.read_edgelist('data/decodedSample.tsv', delimiter='\t', create_using=nx.DiGraph())
#plotting the graph
print(nx.info(subgraph))
pos=nx.spring_layout(subgraph)
nx.draw(subgraph, pos, node_color = '#A0CBE2', edge_color = '#00bb5e', width = 1, edge_cmap = plt.cm.Blues, with_labels = True)
plt.savefig("graph_sample.pdf")

In [ ]:
def generate_missing_edges(out_file, train_percent = 0.7):
    test_percent = 1 - train_percent
    r = pd.read_csv('data/decoded.tsv', delimiter = '\t', header = None)
    nrows = r.shape[0]
    removable_edges = random.sample(range(nrows), int(nrows*test_percent))    
    temp = r
    temp = temp.drop(removable_edges)
    csv = temp.to_csv('data/smaller_edgelist.tsv', sep='\t', index=False, index_label=False)
    pickle.dump(removable_edges,open(out_file, 'wb'))


In [ ]:
generate_missing_edges('data/removedEdgesList.pkl')

In [ ]:
def random_forest_clf(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)
    # print(clf.fit(X_train, y_train).summary())
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return [clf,acc,"Random Forest Classifier"]

def knn_clf(X, y, neighs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
    clf = KNeighborsClassifier(n_neighbors = neighs)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return [clf,acc,"K-Nearest Neighbors"]

In [ ]:
X = []
with open('data/X_data.pkl', 'rb') as f:
    X = pickle.load(f)
y = []
with open('data/Y_data.pkl', 'rb') as f:
    y = pickle.load(f)

In [ ]:
classifiers = []

classifiers.append(random_forest_clf(X,y))

In [ ]:
classifiers[0][1]

In [ ]:
rm_edges = []
with open('data/removedEdgesList.pkl', 'rb') as f:
    rm_edges = pickle.load(f)

# Load original network
G = nx.read_edgelist('data/decoded.tsv', delimiter = '\t', create_using=nx.DiGraph())

# Load smaller network
S = nx.read_edgelist('data/smaller_edgelist.tsv', delimiter = '\t', create_using=nx.DiGraph())

utils.HITS(S, 'data/smaller_hits.pkl')
utils.PR_file(S, 'data/smaller_pr.pkl')

pr = []
with open('data/smaller_pr.pkl', 'rb') as f:
    pr = pickle.load(f)

hits = []
with open('data/smaller_hits.pkl', 'rb') as f:
    hits = pickle.load(f)

cat_dict = []
with open('data/getCategoryFromLinkDict.pkl', 'rb') as f:
    cat_dict = pickle.load(f)

In [ ]:
vecs = []
for e in G.edges():
    u, v = e
    a = list(utils.create_X(S,u,v, cat_dict,pr,hits))
    vec = np.array(a)
    vecs.append(vec)

y_preds = classifiers[0][0].predict(vecs)
score = accuracy_score(np.ones(len(G.edges)),y_preds)
print(score)